In [ ]:
import ast
import pandas as pd
from pathlib import Path
pd.set_option('display.max_colwidth', None)

In [ ]:
# TODO: change me
identity = "disabled people"  # Options: "women", "Muslims", "immigrants", "gays", "trans", "disabled people"

In [ ]:
dataset_path = Path("datasets/gpt3.5-generated/")
functionality_csv_file = dataset_path/"functionalities.csv"
functionality_output_file = dataset_path/f"functionalities_{identity}.csv"
raw_json_file = dataset_path/f"gpt3.5_generated_hs_dataset_{identity}.json"
processed_dataset_file = dataset_path/f"dataset_{identity}.csv"
print(f"raw_json_file: {raw_json_file}")
print(f"processed_dataset_file: {processed_dataset_file}")
print(f"functionality-prompt mapping:{functionality_output_file}")

In [ ]:
# Functionalities independent from identity
independent_functionalities = [
    "F11: Non-hateful use of profanity",
    "F22: Abuse targeted at objects",
    "F23: Abuse targeted at individuals (not as member of a prot. group)",
    "F24: Abuse targeted at non-protected groups (e.g. professions)"
]

## 1. Post-process the generated data

In [ ]:
# load the dataset from file
import ast
with open(raw_json_file, "r") as f:
    data_str = f.read()
    data_dict = ast.literal_eval(data_str)
    
df_dataset = pd.DataFrame(data_dict)
df_dataset.shape

In [ ]:
df_functionalities = pd.read_csv(functionality_csv_file)

In [ ]:
if identity != "black people":
    df_functionalities = df_functionalities[~df_functionalities['functionality'].isin(independent_functionalities)]
    print(f"Totoal {len(df_functionalities)} functionalities")

In [ ]:
assert len(df_functionalities.functionality.tolist()) == len(df_dataset.prompt.unique().tolist())
func2prompt = dict(zip(df_functionalities.functionality.tolist(), df_dataset.prompt.unique().tolist()))
func2prompt

In [ ]:
df_functionalities['prompt'] = df_functionalities.apply(lambda x: func2prompt[x['functionality']], axis=1)

In [ ]:
df_final = pd.merge(df_dataset, df_functionalities, on="prompt", how="inner")
df_final = df_final.drop(columns=['prompt'])
assert len(df_dataset) == len(df_final)

In [ ]:
print(f"Before dropping duplicates {len(df_final)} entries.")
df_final.drop_duplicates(subset=["message"], keep='first', inplace=True, ignore_index=True)
print(f"After dropping duplicates {len(df_final)} entries.")

In [ ]:
df_final.to_csv(processed_dataset_file, index=False, encoding="utf-8")
df_functionalities.to_csv(functionality_output_file, index=False, encoding="utf-8")

### 1.1 Data Exploration

In [ ]:
df = pd.read_csv(processed_dataset_file, encoding="utf-8", low_memory=False)

In [ ]:
df.groupby("functionality").count().reset_index()

In [ ]:
# select the first example for all test cases
df_sample = df.iloc[range(3,len(df),40)]
df_sample